In [9]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from allen_v1dd.client import EMClient
import allen_v1dd.stimulus_analysis.analysis_tools as at

at.set_stylesheet()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
em_client = EMClient()

/opt/anaconda3/envs/allen_v1dd/lib/python3.10/site-packages/standard_transform/datasets.py:24: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, _ = R.align_vectors(np.array([[0, 1, 0]]), [up])


# See table names

In [11]:
em_client.get_tables()

['functional_coregistration_v1',
 'coregistration_landmarks',
 'manual_pilot_functional_coregistration_v1',
 'correspondance_pilot',
 'ariadne_multisoma_proofreading_v1',
 'ariadne_dendrite_proofreading_v2',
 'ariadne_axon_proofreading_v1',
 'synapses_v1dd',
 'multisoma_split_task',
 'ariadne_dendrite_task',
 'ophys_examples',
 'manual_central_types',
 'nucleus_detection_v0',
 'ariadne_axon_task']

In [16]:
em_client.query_table("coregistration_landmarks").drop_duplicates("pt_root_id")

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
0,276266,2022-02-04 06:21:16.129002+00:00,t,377.762618,91491531315515547,864691132827688641,1,2023-04-14 22:08:40.446962+00:00,t,276266,1,3,117,1,NaN,NaN,"[775690, 586656, 193770]","[771111, 581922, 20365]","[780423, 591312, 21369]"
1,276140,2022-02-04 06:30:31.896172+00:00,t,321.738753,92547406008789124,864691132728579766,2,2023-04-14 22:08:40.447864+00:00,t,276140,1,3,75,1,NaN,NaN,"[813015, 598063, 184365]","[808825, 593795, 19385]","[817128, 602564, 20322]"
2,290027,2022-02-04 06:25:08.200797+00:00,t,309.625983,94237492550599227,864691132727393198,3,2023-04-14 22:08:40.448713+00:00,t,290027,1,3,26,1,NaN,NaN,"[871060, 642606, 69930]","[866482, 638260, 7062]","[875716, 646486, 7998]"
3,365414,2022-02-04 06:31:02.914671+00:00,t,363.636744,95855767575844611,864691132782526604,4,2023-04-14 22:08:40.449580+00:00,t,365414,1,3,163,1,NaN,NaN,"[930502, 635544, 110430]","[926078, 630733, 11330]","[935468, 640122, 12489]"
4,365704,2022-02-04 06:21:20.137086+00:00,t,380.395452,97192636343210349,864691132715818816,5,2023-04-14 22:08:40.450423+00:00,t,365704,1,3,31,1,NaN,NaN,"[976518, 630422, 127035]","[971862, 625611, 13177]","[981330, 634846, 14206]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,291956,2022-02-04 06:29:42.120241+00:00,t,203.224374,92618530666908453,864691132687688330,163,2023-04-14 22:08:40.583492+00:00,t,291956,1,3,32,3,NaN,NaN,"[815343, 626698, 172485]","[811696, 622507, 18134]","[819456, 630733, 18997]"
163,279007,2022-02-04 06:27:19.098436+00:00,t,377.945258,93180587669989765,864691132680577619,164,2023-04-14 22:08:40.584262+00:00,t,279007,1,3,250,3,NaN,NaN,"[835209, 593174, 319005]","[830242, 588751, 33780]","[840175, 598374, 34915]"
164,276453,2022-02-04 06:28:39.858694+00:00,t,343.649047,91069250197824154,864691132667436992,165,2023-04-14 22:08:40.585018+00:00,t,276453,1,3,6,1,NaN,NaN,"[760402, 584095, 208350]","[756212, 579750, 22000]","[764981, 588829, 22897]"
165,276650,2022-02-04 06:26:01.052005+00:00,t,348.757005,94025080984162774,864691132575551764,166,2023-04-14 22:08:40.585771+00:00,t,276650,1,3,22,1,NaN,NaN,"[864930, 596744, 206460]","[860817, 592398, 21733]","[869430, 601167, 22727]"


# Query table example: Plot soma positions

In [4]:
nucleus_df = em_client.query_table("nucleus_detection_v0")
nucleus_df["pt_position_microns"] = em_client.transform_position_to_microns(nucleus_df["pt_position"]) # Add a column mapping position to microns
nucleus_df

,id,created,superceded_id,valid,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position,pt_position_microns
0,228132,2022-02-04 06:21:10.292663+00:00,NaN,t,458.464831,87485049993200655,864691132737039043,"[632828, 749849, 738270]","[626620, 746124, 78876]","[638493, 753341, 80137]","[637.4001441732279, 531.6798639403521, 510.601..."
1,543247,2022-02-04 06:21:10.295034+00:00,NaN,t,73.345940,106490931167272264,864691132870078055,"[1304922, 977915, 83880]","[1299179, 974035, 8536]","[1311595, 983192, 9511]","[1309.7076748905974, 572.9044947755802, -180.9..."
2,203262,2022-02-04 06:21:10.297516+00:00,NaN,t,338.276613,87197526360541948,864691132654552792,"[624680, 531094, 283770]","[619869, 525818, 30089]","[629414, 535750, 31050]","[627.5974959640572, 199.28872249700427, 131.24..."
3,350562,2022-02-04 06:21:10.300143+00:00,NaN,t,326.965400,94866276031228266,864691132773514104,"[894573, 478559, 163530]","[891003, 473981, 17111]","[898608, 483370, 18134]","[897.0749802589321, 115.14175620165939, 29.457..."
4,718122,2022-02-04 06:21:10.302911+00:00,NaN,t,333.888647,118515535938912588,864691132774106773,"[1729859, 674111, 781200]","[1722177, 667205, 83852]","[1736610, 682026, 84536]","[1734.1115168756453, 464.7378351221714, 572.23..."
...,...,...,...,...,...,...,...,...,...,...,...
207450,527607,2022-02-04 06:31:31.245454+00:00,NaN,t,100.547645,105284974949287046,864691132639606383,"[1262940, 628094, 734445]","[1258827, 624990, 78856]","[1267053, 630810, 79453]","[1266.9153050750815, 410.20931181865564, 539.5..."
207451,168582,2022-02-04 06:31:31.246802+00:00,NaN,t,369.919126,83482550844689534,864691133042980384,"[491518, 1057067, 92070]","[485388, 1052877, 9399]","[497571, 1061413, 10389]","[496.7042771516083, 655.4081290337385, -194.25..."
207452,29422,2022-02-04 06:31:31.248226+00:00,NaN,t,285.031368,0,0,"[302330, 415005, 81855]","[294182, 409340, 8410]","[308770, 420825, 9099]","[304.4256843161546, 34.99271782807331, -32.224..."
207453,422767,2022-02-04 06:31:31.249663+00:00,NaN,t,394.724290,99723368244567127,864691132851361283,"[1065603, 538932, 36405]","[1061335, 533267, 3434]","[1070182, 544054, 4399]","[1068.2230214645483, 138.43003978714947, -109...."


In [5]:
nucleus_df.drop_duplicates("pt_root_id", inplace=True) # Drop duplicate pt_root_ids
len(nucleus_df)

155652

In [1]:
fig, ax = plt.subplots(figsize=(8, 6))

x_pos = nucleus_df.pt_position_microns.apply(lambda pos: pos[0])
y_pos = nucleus_df.pt_position_microns.apply(lambda pos: pos[1])
z_pos = nucleus_df.pt_position_microns.apply(lambda pos: pos[2])

# Normalize the y points
# q = 0.005
# y_min, y_max = np.quantile(y, [q, 1-q])
# y = (y - y_min) / (y_max - y_min) * y_max

fig.suptitle(f"Nucleus positions for EM cells ($n = {len(nucleus_df)}$)", fontsize=14)
ax.scatter(x_pos, y_pos, color="black", s=0.1, alpha=0.5)
ax.set_xlabel(f"$x$ (µm)", fontsize=14)
ax.set_ylabel("$y$ (cortical depth, µm)", fontsize=14)

ax.axis("equal")
ax.set_ylim(y_pos.min(), y_pos.max())
ax.invert_yaxis() # Open down

text_trans = mpl.transforms.blended_transform_factory(ax.transAxes, ax.transData)

for layer, (depth_min, depth_max) in em_client.layer_boundaries.items():
    ax.axhline(depth_max, xmin=0.02, xmax=0.98, color="red", linestyle="dashed")
    ax.text(1, depth_max, f"{depth_max} µm", ha="left", va="center", color="red", fontsize=10, transform=text_trans)
    text_depth = depth_max - 60 if np.isinf(depth_min) else (depth_min + depth_max) / 2
    ax.text(1, text_depth, f"L{layer}", ha="left", va="center", color="darkred", fontweight="bold", transform=text_trans)

NameError: name 'plt' is not defined

# Count cells with proofreading

In [12]:
axo_proof_df = em_client.get_axon_proofreading_table()
den_proof_df = em_client.get_dendrite_proofreading_table()

In [13]:
print("Axon:")
print(axo_proof_df.cell_type.value_counts())
print("\nDendrite:")
print(den_proof_df.cell_type.value_counts())

Axon:
submitted    808
clean         62
complete       9
Name: cell_type, dtype: int64

Dendrite:
submitted    2420
complete       13
Name: cell_type, dtype: int64
